# 0.743

In [ ]:
!pip install catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import warnings
import random
import os
plt.style.use('ggplot')
warnings.filterwarnings(action='ignore')

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/train.csv')
test = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/test.csv')

In [ ]:
col_list = train.columns
nan_list = []
nan_cnt = []
nan_col = []
full_list = []
for col in col_list:
    if train[col].isnull().sum() == 0 :
        full_list.append(col)
        continue
    nan_list.append([col, train[col].isnull().sum()])
    nan_cnt.append(train[col].isnull().sum())
    nan_col.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_list :
    if col[1] == 598 :
        del_col.append(col[0])
train = train.drop(columns=del_col)
test = test.drop(columns=del_col)
train.head(3)

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN


In [ ]:
trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

In [ ]:
col_list = train.columns
nan_listA_31 = []
nan_cntA_31 = []
nan_colA_31 = []
full_listA_31 = []
for col in col_list:
    if trainA_31[col].isnull().sum() == 0 :
        full_listA_31.append(col)
        continue
    nan_listA_31.append([col, trainA_31[col].isnull().sum()])
    nan_cntA_31.append(trainA_31[col].isnull().sum())
    nan_colA_31.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listA_31 :
    if col[1] == len(trainA_31) :
        del_col.append(col[0])
trainA_31 = trainA_31.drop(columns=del_col)
testA_31 = testA_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = trainA_31.columns
for col in col_list[6:] :
    if trainA_31[col].nunique()==1 :
        del_col.append(col)
trainA_31 = trainA_31.drop(columns=del_col)
testA_31 = testA_31.drop(columns=del_col)

In [ ]:
col_list = train.columns
nan_listO = []
nan_cntO = []
nan_colO = []
full_listO = []
for col in col_list:
    if train_O_31[col].isnull().sum() == 0 :
        full_listO.append(col)
        continue
    nan_listO.append([col, train_O_31[col].isnull().sum()])
    nan_cntO.append(train_O_31[col].isnull().sum())
    nan_colO.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listO :
    if col[1] == len(train_O_31) :
        del_col.append(col[0])
train_O_31 = train_O_31.drop(columns=del_col)
test_O_31 = test_O_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_O_31.columns
for col in col_list[6:] :
    if train_O_31[col].nunique()==1 :
        del_col.append(col)
train_O_31 = train_O_31.drop(columns=del_col)
test_O_31 = test_O_31.drop(columns=del_col)

In [ ]:
col_list = train.columns
nan_listT = []
nan_cntT = []
nan_colT = []
full_listT = []
for col in col_list:
    if train_T_31[col].isnull().sum() == 0 :
        full_listT.append(col)
        continue
    nan_listT.append([col, train_T_31[col].isnull().sum()])
    nan_cntT.append(train_T_31[col].isnull().sum())
    nan_colT.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listT :
    if col[1] == len(train_T_31) :
        del_col.append(col[0])
train_T_31 = train_T_31.drop(columns=del_col)
test_T_31 = test_T_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_T_31.columns
for col in col_list[6:] :
    if train_T_31[col].nunique()==1 :
        del_col.append(col)
train_T_31 = train_T_31.drop(columns=del_col)
test_T_31 = test_T_31.drop(columns=del_col)

In [ ]:
trainA_31_x = trainA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
testA_31_x = testA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_T_31_x = train_T_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','PRODUCT_CODE'])
test_T_31_x = test_T_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_O_31_x = train_O_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
test_O_31_x = test_O_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])

# classification
trainA_31_y_c = trainA_31['Y_Class']
train_T_31_y_c = train_T_31['Y_Class']
train_O_31_y_c = train_O_31['Y_Class']

# regression
trainA_31_y_r = trainA_31['Y_Quality']
train_T_31_y_r = train_T_31['Y_Quality']
train_O_31_y_r = train_O_31['Y_Quality']

In [ ]:
print(trainA_31_x.shape,'\n',testA_31_x.shape,'\n',train_T_31_x.shape,'\n',test_T_31_x.shape,'\n',train_O_31_x.shape,'\n',test_O_31_x.shape)

(249, 1866) 
 (67, 1866) 
 (343, 550) 
 (239, 550) 
 (6, 499) 
 (4, 499)


In [ ]:
del_cols = trainA_31_x.T[trainA_31_x.T.duplicated()].index

trainA_31_x = trainA_31_x.drop(columns=del_cols)
testA_31_x = testA_31_x.drop(columns=del_cols)

In [ ]:
trainA_31_x['LINE'].unique()

array(['T050304', 'T050307', 'T010306', 'T010305'], dtype=object)

In [ ]:
testA_31_x['LINE'].unique()

array(['T010305', 'T010306', 'T050304', 'T050307'], dtype=object)

In [ ]:
print(len(trainA_31_x[trainA_31_x['LINE']==0]),'\n',
len(trainA_31_x[trainA_31_x['LINE']==1]),'\n',
len(trainA_31_x[trainA_31_x['LINE']==2]),'\n',
len(trainA_31_x[trainA_31_x['LINE']==3]),'\n',
len(testA_31_x[testA_31_x['LINE']==0]),'\n',
len(testA_31_x[testA_31_x['LINE']==1]),'\n',
len(testA_31_x[testA_31_x['LINE']==2]),'\n',
len(testA_31_x[testA_31_x['LINE']==3]))

0 
 0 
 0 
 0 
 0 
 0 
 0 
 0


In [ ]:
trainA_31_x=trainA_31_x.fillna(-1)
testA_31_x=testA_31_x.fillna(-1)
train_T_31_x=train_T_31_x.fillna(-1)
test_T_31_x=test_T_31_x.fillna(-1)
train_O_31_x=train_O_31_x.fillna(-1)
test_O_31_x=test_O_31_x.fillna(-1)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(trainA_31_x[i])
    trainA_31_x[i] = le.transform(trainA_31_x[i])
    
    for label in np.unique(testA_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    testA_31_x[i] = le.transform(testA_31_x[i]) 
print('Done.')

Done.


In [ ]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_T_31_x[i])
    train_T_31_x[i] = le.transform(train_T_31_x[i])
    
    for label in np.unique(test_T_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_T_31_x[i] = le.transform(test_T_31_x[i]) 
print('Done.')

Done.


In [ ]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_O_31_x[i])
    train_O_31_x[i] = le.transform(train_O_31_x[i])
    
    for label in np.unique(test_O_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_O_31_x[i] = le.transform(test_O_31_x[i]) 
print('Done.')

Done.


In [ ]:
#from imblearn.combine import SMOTETomek

In [ ]:
from catboost import *

In [ ]:
model = CatBoostRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033)
model.fit(trainA_31_x, trainA_31_y_r)
pred_a = model.predict(testA_31_x)

0:	learn: 0.0098361	total: 248ms	remaining: 6m 11s
500:	learn: 0.0006463	total: 43.8s	remaining: 1m 27s
1000:	learn: 0.0000602	total: 1m 18s	remaining: 39s
1499:	learn: 0.0000055	total: 1m 53s	remaining: 0us


In [ ]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(train_T_31_x, train_T_31_y_r)
pred_t = model.predict(test_T_31_x)

0:	learn: 0.0047159	total: 47.6ms	remaining: 1m 11s
500:	learn: 0.0005292	total: 18.7s	remaining: 37.3s
1000:	learn: 0.0000593	total: 36.7s	remaining: 18.3s
1499:	learn: 0.0000079	total: 54.8s	remaining: 0us


In [ ]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(train_O_31_x, train_O_31_y_r)
pred_o = model.predict(test_O_31_x)

0:	learn: 0.0031679	total: 4.12ms	remaining: 6.18s
500:	learn: 0.0000479	total: 1.75s	remaining: 3.49s
1000:	learn: 0.0000008	total: 3.31s	remaining: 1.65s
1499:	learn: 0.0000000	total: 4.83s	remaining: 0us


In [ ]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

In [ ]:
submita = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')
submitt = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')
submito = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')

In [ ]:
submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_quanlity']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_quanlity']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_quanlity']],on='PRODUCT_ID')

In [ ]:
pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('product_code.csv',index=False)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/train.csv')
test = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/test.csv')

In [ ]:
col_list = train.columns
nan_list = []
nan_cnt = []
nan_col = []
full_list = []
for col in col_list:
    if train[col].isnull().sum() == 0 :
        full_list.append(col)
        continue
    nan_list.append([col, train[col].isnull().sum()])
    nan_cnt.append(train[col].isnull().sum())
    nan_col.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_list :
    if col[1] == 598 :
        del_col.append(col[0])
train = train.drop(columns=del_col)
test = test.drop(columns=del_col)
train.head(3)

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN


In [ ]:
train['LINE_header'] = train['LINE'].map(lambda x: x[:3])
test['LINE_header'] = test['LINE'].map(lambda x: x[:3])

In [ ]:
train_T01 = train[train['LINE_header']=='T01']
train_T05 = train[train['LINE_header']=='T05']
train_T10 = train[train['LINE_header']=='T10']

test_T01 = test[test['LINE_header']=='T01']
test_T05 = test[test['LINE_header']=='T05']
test_T10 = test[test['LINE_header']=='T10']

In [ ]:
col_list = train.columns
nan_listA_31 = []
nan_cntA_31 = []
nan_colA_31 = []
full_listA_31 = []
for col in col_list:
    if train_T01[col].isnull().sum() == 0 :
        full_listA_31.append(col)
        continue
    nan_listA_31.append([col, train_T01[col].isnull().sum()])
    nan_cntA_31.append(train_T01[col].isnull().sum())
    nan_colA_31.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listA_31 :
    if col[1] == len(train_T01) :
        del_col.append(col[0])
train_T01 = train_T01.drop(columns=del_col)
test_T01 = test_T01.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_T01.columns
for col in col_list[6:] :
    if train_T01[col].nunique()==1 :
        del_col.append(col)
train_T01 = train_T01.drop(columns=del_col)
test_T01 = test_T01.drop(columns=del_col)

In [ ]:
col_list = train.columns
nan_listO = []
nan_cntO = []
nan_colO = []
full_listO = []
for col in col_list:
    if train_T10[col].isnull().sum() == 0 :
        full_listO.append(col)
        continue
    nan_listO.append([col, train_T10[col].isnull().sum()])
    nan_cntO.append(train_T10[col].isnull().sum())
    nan_colO.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listO :
    if col[1] == len(train_T10) :
        del_col.append(col[0])
train_T10 = train_T10.drop(columns=del_col)
test_T10 = test_T10.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_T10.columns
for col in col_list[6:] :
    if train_T10[col].nunique()==1 :
        del_col.append(col)
train_T10 = train_T10.drop(columns=del_col)
test_T10 = test_T10.drop(columns=del_col)

In [ ]:
col_list = train.columns
nan_listT = []
nan_cntT = []
nan_colT = []
full_listT = []
for col in col_list:
    if train_T05[col].isnull().sum() == 0 :
        full_listT.append(col)
        continue
    nan_listT.append([col, train_T05[col].isnull().sum()])
    nan_cntT.append(train_T05[col].isnull().sum())
    nan_colT.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listT :
    if col[1] == len(train_T05):
        del_col.append(col[0])
train_T05 = train_T05.drop(columns=del_col)
test_T05 = test_T05.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_T05.columns
for col in col_list[6:] :
    if train_T05[col].nunique()==1 :
        del_col.append(col)
train_T05 = train_T05.drop(columns=del_col)
test_T05 = test_T05.drop(columns=del_col)

In [ ]:
train_T01_x = train_T01.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
test_T01_x = test_T01.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_T05_x = train_T05.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','PRODUCT_CODE'])
test_T05_x = test_T05.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_T10_x = train_T10.drop(columns=['PRODUCT_ID','TIMESTAMP', 'Y_Class','Y_Quality'])
test_T10_x = test_T10.drop(columns=['PRODUCT_ID','TIMESTAMP'])

# classification
train_T01_y_c = train_T01['Y_Class']
train_T05_y_c = train_T05['Y_Class']
train_T10_y_c = train_T10['Y_Class']

# regression
train_T01_y_r = train_T01['Y_Quality']
train_T05_y_r = train_T05['Y_Quality']
train_T10_y_r = train_T10['Y_Quality']

In [ ]:
print(train_T01_x.shape,'\n',test_T01_x.shape,'\n',train_T05_x.shape,'\n',test_T05_x.shape,'\n',train_T10_x.shape,'\n',test_T10_x.shape)

(129, 746) 
 (28, 746) 
 (120, 1754) 
 (39, 1754) 
 (349, 553) 
 (243, 553)


In [ ]:
train_T01_x['LINE'].unique()

array(['T010306', 'T010305'], dtype=object)

In [ ]:
test_T01_x['LINE'].unique()

array(['T010305', 'T010306'], dtype=object)

In [ ]:
print(len(train_T01_x[train_T01_x['LINE']==0]),'\n',
len(train_T01_x[train_T01_x['LINE']==1]),'\n',
len(train_T01_x[train_T01_x['LINE']==2]),'\n',
len(train_T01_x[train_T01_x['LINE']==3]),'\n',
len(test_T01_x[test_T01_x['LINE']==0]),'\n',
len(test_T01_x[test_T01_x['LINE']==1]),'\n',
len(test_T01_x[test_T01_x['LINE']==2]),'\n',
len(test_T01_x[test_T01_x['LINE']==3]))

0 
 0 
 0 
 0 
 0 
 0 
 0 
 0


In [ ]:
train_T01_x=train_T01_x.fillna(-1)
test_T01_x=test_T01_x.fillna(-1)
train_T05_x=train_T05_x.fillna(-1)
test_T05_x=test_T05_x.fillna(-1)
train_T10_x=train_T10_x.fillna(-1)
test_T10_x=test_T10_x.fillna(-1)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_T01_x[i])
    train_T01_x[i] = le.transform(train_T01_x[i])
    
    for label in np.unique(test_T01_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_T01_x[i] = le.transform(test_T01_x[i]) 
print('Done.')

Done.


In [ ]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_T05_x[i])
    train_T05_x[i] = le.transform(train_T05_x[i])
    
    for label in np.unique(test_T05_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_T05_x[i] = le.transform(test_T05_x[i]) 
print('Done.')

Done.


In [ ]:
# qualitative to quantitative
qual_col = ['LINE','PRODUCT_CODE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_T10_x[i])
    train_T10_x[i] = le.transform(train_T10_x[i])
    
    for label in np.unique(test_T10_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_T10_x[i] = le.transform(test_T10_x[i]) 
print('Done.')

Done.


In [ ]:
#from imblearn.combine import SMOTETomek

In [ ]:
from catboost import *

In [ ]:
model = CatBoostRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033)
model.fit(train_T01_x, train_T01_y_r)
pred_a = model.predict(test_T01_x)

0:	learn: 0.0089365	total: 64.1ms	remaining: 1m 36s
500:	learn: 0.0001748	total: 19.7s	remaining: 39.2s
1000:	learn: 0.0000031	total: 38.8s	remaining: 19.3s
1499:	learn: 0.0000000	total: 58s	remaining: 0us


In [ ]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(train_T05_x, train_T05_y_r)
pred_t = model.predict(test_T05_x)

0:	learn: 0.0106083	total: 104ms	remaining: 2m 36s
500:	learn: 0.0001807	total: 34.5s	remaining: 1m 8s
1000:	learn: 0.0000032	total: 1m 10s	remaining: 35.3s
1499:	learn: 0.0000001	total: 1m 45s	remaining: 0us


In [ ]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(train_T10_x, train_T10_y_r)
pred_o = model.predict(test_T10_x)

0:	learn: 0.0046968	total: 66.4ms	remaining: 1m 39s
500:	learn: 0.0005441	total: 18.8s	remaining: 37.6s
1000:	learn: 0.0000573	total: 37.5s	remaining: 18.7s
1499:	learn: 0.0000063	total: 56.3s	remaining: 0us


In [ ]:
test_T01['Y_quanlity'] = pred_a
test_T05['Y_quanlity'] = pred_t
test_T10['Y_quanlity'] = pred_o

In [ ]:
submita = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')
submitt = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')
submito = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')

In [ ]:
submita = pd.merge(submita[['PRODUCT_ID']],test_T01[['PRODUCT_ID','Y_quanlity']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T05[['PRODUCT_ID','Y_quanlity']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_T10[['PRODUCT_ID','Y_quanlity']],on='PRODUCT_ID')

In [ ]:
pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('line.csv',index=False)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')
submit1 = pd.read_csv('product_code.csv')
submit2 = pd.read_csv('line.csv')

In [ ]:
submit['Y_quanlity'] = (submit1['Y_quanlity'] + submit2['Y_quanlity']) / 2
submit.head()

,PRODUCT_ID,Y_Class,Y_quanlity
0,TEST_000,0,0.529790
1,TEST_001,0,0.532917
2,TEST_002,0,0.531867
3,TEST_003,0,0.524574
4,TEST_004,0,0.531313


In [ ]:
submit['Y_Class'] = 1

submit.loc[(submit['Y_quanlity']<0.52507), 'Y_Class'] = 0
submit.loc[(submit['Y_quanlity']>0.5349), 'Y_Class'] = 2

In [ ]:
submit = submit.drop(columns='Y_quanlity')
submit.to_csv('캣2_5.csv',index=False)

In [ ]:
submit.head()

,PRODUCT_ID,Y_Class
0,TEST_000,1
1,TEST_001,1
2,TEST_002,1
3,TEST_003,0
4,TEST_004,1


In [ ]:
submit['Y_Class'].value_counts()

1    283
0     22
2      5
Name: Y_Class, dtype: int64

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

# (0.8928055698813822, 0.9741935483870968)
best = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/캣2_5.csv')
f1_score(best['Y_Class'], submit['Y_Class'], average='macro'), accuracy_score(best['Y_Class'], submit['Y_Class'])

(0.8760775664001471, 0.967741935483871)